# 1. Основы PySpark

## 1.1. Установка и архитектура

### 1.1.1. Установка PySpark (pip) + Java

Создание виртуального окружения (Python 3.11 гарантирует полную совместимость с PySpark без дополнительных костылей)

Установите Python 3.11 если еще нет:  
https://www.python.org/downloads/release/python-3119/  

*powershell*  
`py -3.11 -m venv pyspark_env`  
`pyspark_env\Scripts\activate`  

Установка PySpark и зависимостей

*powershell*  
`pip install pyspark==3.5.0 jupyter pandas pyarrow`

Проверочный код (в первой ячейке Jupyter)

In [1]:
from pyspark.sql import SparkSession
import sys

# Проверка версий
print(f"Python: {sys.version}")
print(f"VenV: {sys.prefix != sys.base_prefix}")  # Проверим, что в VENV

# Инициализация Spark
spark = SparkSession.builder \
    .appName("PySpark 3.11 Test") \
    .getOrCreate()

# Тестовый расчет
df = spark.range(1, 5)
df.selectExpr("id * 10 as value").show()

Python: 3.11.9 (tags/v3.11.9:de54cf5, Apr  2 2024, 10:12:12) [MSC v.1938 64 bit (AMD64)]
VenV: True
+-----+
|value|
+-----+
|   10|
|   20|
|   30|
|   40|
+-----+



1. Создайте DataFrame с колонками:
- user_id (integer)
- action (string: "login", "purchase", "logout")
- timestamp (current time)

2. Используйте withColumn для добавления колонки is_purchase (boolean)

3. Отфильтруйте только события покупки

In [2]:
from pyspark.sql import functions as F
from datetime import datetime

# Создаем тестовые данные
data = [
    (1, "login", datetime.now()),
    (1, "purchase", datetime.now()),
    (2, "purchase", datetime.now()),
    (2, "logout", datetime.now())
]

df = spark.createDataFrame(data, ["user_id", "action", "timestamp"])

# Добавляем колонку is_purchase
df = df.withColumn("is_purchase", F.col("action") == "purchase")

# Фильтрация
purchases = df.filter(F.col("is_purchase"))

# Результат
print("Все события:")
display(df)

print("\nТолько покупки:")
display(purchases)

d = [{'name': 'Alice', 'age': 1}]
spark.createDataFrame(d).show()

Все события:


DataFrame[user_id: bigint, action: string, timestamp: timestamp, is_purchase: boolean]


Только покупки:


DataFrame[user_id: bigint, action: string, timestamp: timestamp, is_purchase: boolean]

Py4JJavaError: An error occurred while calling o80.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 16) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:364)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:328)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadChe

### 1.1.2. Настройка Jupyter Notebook

### 1.1.3. Архитектура Spark (Driver, Executor, Cluster Manager)

### 1.1.4. RDD vs DataFrame vs Dataset

### 1.1.5. Практика: Создание SparkSession

## 1.2. Работа с DataFrame API